In [ ]:
!pip install fuzzywuzzy python-Levenshtein
# Basic text processing
import re
import string


# NLP libraries
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
from fuzzywuzzy import process

# SpaCy for advanced NLP
import spacy

# Text preprocessing with scikit-learn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Optional: pandas and numpy for data handling
import pandas as pd
import numpy as np


In [ ]:
#--- Tokenization with CamelCase Support ---
def tokenize_enhanced(text):
    if pd.isnull(text):
        return []
    spaced = re.sub('([a-z])([A-Z])', r'\1 \2', text)
    return re.findall(r'\b\w+\b', spaced.lower())


In [ ]:
# --- Load Data ---
df = pd.read_excel("/content/GP LP mapping list.xlsx", sheet_name='LP')
print (df.head())

     Region                       LP  Match                    Company Name  \
0  Americas                     1864    NaN                    1010 Capital   
1      APAC       1010 Family Office    NaN  1199SEIU National Benefit Fund   
2    Europe                   Abbott    NaN                          123 IM   
3    Europe                      ABN    NaN                 1693 Management   
4        ME  Abu Dhabi Capital Group    NaN                   1823 Partners   

     Country  
0  Australia  
1        USA  
2     France  
3        USA  
4        USA  


In [ ]:
def build_token_frequency(company_names):
    tokenized = company_names.dropna().apply(tokenize_enhanced)
    tokens = [token for sublist in tokenized for token in sublist]
    return Counter(tokens)

In [ ]:
def improved_token_match(lp_value, choices, token_freq):
    if pd.isnull(lp_value):
        return None, None

    lp_tokens = tokenize_enhanced(lp_value)
    lp_nums = [t for t in lp_tokens if t.isdigit()]
    lp_words = [t for t in lp_tokens if not t.isdigit()]

    best_match = None
    best_score = 0

    for company in choices:
        company_tokens = tokenize_enhanced(company)
        company_nums = [t for t in company_tokens if t.isdigit()]
        numeric_overlap = len(set(lp_nums) & set(company_nums))

        if numeric_overlap > 0:
            score = 1000 * numeric_overlap
        else:
            score = sum(token_freq[token] for token in lp_words if token in company_tokens)

        if score > best_score:
            best_score = score
            best_match = company

    return best_match, min(100, best_score) if best_score > 0 else None

# --- Highlight Inaccurate Matches ---
def is_inaccurate(lp, match):
    if pd.isnull(lp) or pd.isnull(match):
        return True
    lp_tokens = set(tokenize_enhanced(lp))
    match_tokens = set(tokenize_enhanced(match))
    return len(lp_tokens & match_tokens) == 0


In [ ]:
def process_matching(df):
    company_names = df['Company Name'].dropna().unique().tolist()
    token_freq = build_token_frequency(df['Company Name'])
    results = df['LP'].apply(lambda x: improved_token_match(x, company_names, token_freq))
    df['Match'] = results.apply(lambda x: x[0])
    df['Score %'] = results.apply(lambda x: x[1])
    df['Flag'] = df.apply(lambda row: 'Check' if is_inaccurate(row['LP'], row['Match']) else '', axis=1)
    return df[['LP', 'Match', 'Score %', 'Flag', 'Company Name']]

In [ ]:
# Process the matching
processed_df = process_matching(df)


# Display in notebook
print(processed_df)



                           LP                    Match  Score %   Flag  \
0                        1864            1864 Holdings    100.0          
1          1010 Family Office             1010 Capital    100.0          
2                      Abbott           Abbott Capital      2.0          
3                         ABN                 ABN AMRO      2.0          
4     Abu Dhabi Capital Group  Abu Dhabi Capital Group    100.0          
...                       ...                      ...      ...    ...   
3059                      NaN                     None      NaN  Check   
3060                      NaN                     None      NaN  Check   
3061                      NaN                     None      NaN  Check   
3062                      NaN                     None      NaN  Check   
3063                      NaN                     None      NaN  Check   

                        Company Name  
0                       1010 Capital  
1     1199SEIU National Benefit F

In [ ]:
# Export to Excel file
output_path = 'matched_output.xlsx'
processed_df.to_excel(output_path, index=False)

print(f"Matching results saved to {output_path}")


Matching results saved to matched_output.xlsx


In [ ]:
def run_lp_matching(input_path, sheet_name='LP', output_path='matched_output.xlsx'):
    df = pd.read_excel(input_path, sheet_name=sheet_name)
    processed_df = process_matching(df)
    processed_df.to_excel(output_path, index=False)
    print(f"Matching results saved to: {output_path}")

In [ ]:
run_lp_matching('/content/GP LP mapping list.xlsx', sheet_name='LP', output_path='matched_output.xlsx')


Matching results saved to: matched_output.xlsx
